In [19]:
import json
import numpy as np
import os
import pandas as pd
import pathlib

In [20]:
JSON_DIR_PATH = 'data/output/json'
PAID_DATE = 'paid_date'
CATEGORY = 'income'
BASE_DIR = '/Users/dev-yoshiki/Hobby/paycheck_reader'
EMP_NO = 9999999
JSON_DIR_PATH = '/Users/dev-yoshiki/Hobby/paycheck_reader/data/output/json'
JSON_FILES = ['1300_1614120_K_20180425.json', '1300_1614120_K_20180525.json', '1300_1614120_K_20180625.json', '1300_1614120_K_20180725.json', '1300_1614120_K_20180824.json', '1300_1614120_K_20180925.json', '1300_1614120_K_20181025.json', '1300_1614120_K_20181122.json', '1300_1614120_K_20181225.json', '1300_1614120_K_20190125.json', '1300_1614120_K_20190225.json', '1300_1614120_K_20190325.json', '1300_1614120_K_20190425.json', '1300_1614120_K_20190524.json', '1300_1614120_K_20190625.json', '1300_1614120_K_20190725.json', '1300_1614120_K_20190823.json', '1300_1614120_K_20190925.json', '1300_1614120_K_20191025.json', '1300_1614120_K_20191125.json', '1300_1614120_K_20191225.json', '1300_1614120_K_20200124.json', '1300_1614120_K_20200225.json']

In [21]:
dataframes = []
for filename in JSON_FILES:
    dates, keys, values, indexes = [], [], [], []

    file_path = pathlib.Path(JSON_DIR_PATH, filename)
    with open(file_path, 'r') as json_file:
        dict_data = json.load(json_file)

    """Single key extraction"""
    dates, keys, values = [], [], []
    date = dict_data[PAID_DATE]
    for key, value in dict_data['incomes'].items():
        values.append(value)
        keys.append(key)
        dates.append(date)
    df = pd.DataFrame({'date': dates, 'type': keys, 'value': values})
    dataframes.append(df)

df = pd.concat(dataframes)

In [22]:
# Sort data by ranking values statistically.
table = pd.pivot_table(df, index='type', columns='date', values='value', fill_value=0)
table['total'] = table.filter(regex='\d{4}-\d{2}-\d{2}').sum(axis=1)
table['mean'] = table.filter(regex='\d{4}-\d{2}-\d{2}').mean(axis=1)
table['std'] = table.filter(regex='\d{4}-\d{2}-\d{2}').std(axis=1)
# table[['total'], ['mean'], ['std']]
table[['total'], ['mean']]
# table

TypeError: '(['total'], ['mean'])' is an invalid key

In [ ]:
table[['mean'],['total']]

In [30]:
newTable = table.transpose()
EN_TYPE = ['Alfa', 'Bravo', 'Charlie', 'Delta', 'Echo', 'Foxtrot',
        'Golf', 'Hotel', 'India', 'Juliett', 'Kilo', 'LIma', 'Mike', 
        'November', 'Oscar', 'Papa', 'Quebec', 'Romeo', 'Sierra']

col_num = len(table.columns)
EN_TYPE = EN_TYPE[:col_num]
col_dict = dict(zip(table.columns, EN_TYPE))
newTable = table.rename(columns=col_dict)
newTable

date,Alfa,Bravo,Charlie,Delta,Echo,Foxtrot,Golf,Hotel,India,Juliett,...,November,Oscar,Papa,Quebec,Romeo,Sierra,2019-11-25,2019-12-25,2020-01-24,2020-02-25
type,,,,,,,,,,,,,,,,,,,,,
Smart&Fun!支援金,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,...,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000
その他支給(課税加算),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,50000,0,0
不足日数時間精算,0,0,0,0,-13448,-12944,-2365,-12222,0,0,...,0,0,0,0,0,0,0,0,0,0
休日勤務手当,0,0,0,15833,15833,0,15833,0,0,0,...,0,0,4086,0,0,0,0,0,0,0
基本給,220000,220000,220000,220000,220000,220000,220000,220000,220000,220000,...,220000,224500,224500,224500,224500,224500,224500,224500,224500,224500
新卒住宅補助,0,0,0,25000,25000,25000,25000,25000,25000,25000,...,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000
時間外勤務手当,0,1833,0,20158,73300,64138,64138,75133,60473,23823,...,72345,32152,45390,58629,49173,47282,41608,49173,13239,1892
深夜勤務手当,0,0,0,0,0,367,0,0,367,0,...,742,379,3026,757,0,379,0,757,0,0
資格取得(奨励金),0,0,0,25700,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10000
